<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/03_langchain/05_introduction_to_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env' # read local .env file
load_dotenv(dotenv_path)

import warnings
warnings.filterwarnings("ignore")

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers.

In [ ]:
llm_model = "gpt-4o-mini"

## Built-in LangChain tools

In [ ]:
!pip install -U wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=d95166242dd68daab21dd8fe34ba00108816dd73f110e2ff4f3ea3044bd60f28
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [ ]:
!pip install langchain langchain-experimental langchain-openai langchain-community

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

/tmp/ipython-input-1680930130.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent= initialize_agent(


In [ ]:
agent("What is the 25% of 300?")

/tmp/ipython-input-1453878245.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent("What is the 25% of 300?")




> Entering new AgentExecutor chain...
Thought: To find 25% of 300, I need to calculate 25/100 multiplied by 300. This can be done using a calculator. 
Action:
```
{
  "action": "Calculator",
  "action_input": "25/100 * 300"
}
```

Observation: Answer: 75.0
Thought:I now know the final answer.  
Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

## Wikipedia example

In [ ]:
question = "Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). What book did he write?"
result = agent(question)



> Entering new AgentExecutor chain...
Thought: I need to find out what book Tom M. Mitchell wrote. I will look up his works to find the relevant information.
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. I

## Python Agent

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
agent.invoke(f"""Sort these customers by last name and then first name and print the output: {customer_list}""")



> Entering new AgentExecutor chain...
To sort the list of customers by last name and then by first name, I will use Python's built-in sorting capabilities. I will sort the list of lists using a custom key that sorts primarily by the last name and secondarily by the first name. 

Action: Python_REPL  
Action Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]; sorted_customers = sorted(customers, key=lambda x: (x[0], x[1])); print(sorted_customers)`  
Observation: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

Thought:I now know the final answer
Final Answer: [['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]

> Finished chain.


{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "[['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Harrison', 'Chase'], ['Jen', 'Ayai'], ['Lang', 'Chain'], ['Trance', 'Former']]"}

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True
agent.invoke(f"""Sort these customers by last name and then first name and print the output: {customer_list}""")
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

## Define a custom tool

In [ ]:
#!pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any questions related to knowing todays date. The input should always be an empty string, and this function will always return todays date - any date mathmatics should occur outside this function."""
    return str(date.today())

In [ ]:
agent= initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**:

The agent will sometimes come to the wrong conclusion (agents are a work in progress!).

If it does, we rerun it.

In [ ]:
try:
    result = agent("what's the date today?")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I need to find out today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-11-13
Thought:I now know the final answer.
Final Answer: 2025-11-13

> Finished chain.


In this notebook, we began building simple AI agents using both ```langchain.agents``` (via ```initialize_agent(...)```) and ```langchain_experimental.agents.agent_toolkits``` (via ```create_python_agent(...)```). We combined ready-made tools from ```langchain.agents``` such as ```load_tools(["llm-math", "wikipedia"], llm=llm)``` with a simple custom tool to extend our agent's capabilities.

This was just a soft introduction to those foundations, as today's agents are far more advanced and optimized, leveraging modern frameworks like **LangGraph** and **CrewAI** to perform multi-step reasoning, collaboration, and fully autonomous workflows.